# Import de Librairies pour entraîner UNet

**Objectif:** le but de ce notebook est de montrer comment importer les différentes librairies python pour entraîner le modèle UNet. Ce notebook est une version plus 'soft' du notebook précédent "Example_all_steps_UNet_Inria_pytorch".

### Root Variables 

In [1]:
import os  

In [2]:
root = '/home/ign.fr/ttea/Code_IGN/AerialImageDataset'
train_dir = os.path.join(root,'train/images')
gt_dir = os.path.join(root,'train/gt')
test_dir = os.path.join(root,'test/images')

In [3]:
import sys

In [4]:
sys.path.insert(0, '/home/ign.fr/ttea/stage_segmentation_2021/Code')

In [5]:
from dataloader.dataloader import InriaDataset
from model.model import UNet
from train import train, eval, train_full

### Import Libraries 

In [6]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd 

In [7]:
var= pd.read_json('variables.json')

## Dataset

In [8]:
tile_size = (512,512)
train_dataset = InriaDataset(var['variables']['root'],tile_size,'train',None,False,1)
val_dataset = InriaDataset(var['variables']['root'],tile_size,'validation',None,False,1)

## U-Net Model

![title](../img/Unet.png)

### Arguments & Hyperparamètres 

In [10]:
hparam = {
    'lr':0.0001,
    'n_epoch':5,
    'n_epoch_test':int(5),
    'n_class':1,
    'batch_size':8,
    'n_channel':3,
    'conv_width':[16,32,64,128,256,128,64,32,16],
}

In [15]:
tile_size = (512,512)

weights = [0.5, 1.0]
class_weights = torch.FloatTensor(weights).cuda()

args = {
    'nn_loss':nn.BCEWithLogitsLoss(reduction="mean"),
    #'nn_loss':nn.CrossEntropyLoss(weight = class_weights,reduction="mean"),
    #'nn_loss':BinaryDiceLoss,
     'loss_name': 'BinaryCrossentropy',
    # 'loss_name': 'Crossentropy',
    #'loss_name':'BinaryDiceLoss',
    'threshold':0.5,
    'cuda':1,
    'class_names':['None','Batiment'],
    'save_model':False,
    'save_model_name':"unet_test_8_1.pth",
    'train_dataset':InriaDataset(var['variables']['root'],tile_size,'train',None,False,1),
    'val_dataset':InriaDataset(var['variables']['root'],tile_size,'validation',None,False,1),
}

In [16]:
model = UNet(hparam['n_channel'], hparam['conv_width'], hparam['n_class'], cuda=args['cuda'])

### Entraînement du modèle UNet 

In [17]:
trained_model,  metrics_train, metrics_test = train_full(args, model,hparam['lr'],hparam['n_epoch'],
                                    hparam['n_epoch_test'],hparam['batch_size'],hparam['n_class'],
                                    hparam['n_channel'])

Total number of parameters: 1941105


  0%|                                                                                                         …

None : 86.85%  |  Batiment : 29.60%
Epoch   0 -> Train Overall Accuracy: 87.54% Train mIoU : 58.23% Train Loss: 0.3281
None : 86.85%  |  Batiment : 29.60%


  0%|                                                                                                         …

None : 90.25%  |  Batiment : 50.46%
Epoch   1 -> Train Overall Accuracy: 91.13% Train mIoU : 70.36% Train Loss: 0.2161
None : 90.25%  |  Batiment : 50.46%


...

None : 92.07%  |  Batiment : 60.23%
Epoch   3 -> Train Overall Accuracy: 92.92% Train mIoU : 76.15% Train Loss: 0.1734
None : 92.07%  |  Batiment : 60.23%


  0%|                                                                                                         …

None : 92.53%  |  Batiment : 62.66%
Epoch   4 -> Train Overall Accuracy: 93.36% Train mIoU : 77.59% Train Loss: 0.1629
None : 92.53%  |  Batiment : 62.66%


  0%|                                                                                                         …

None : 91.35%  |  Batiment : 48.77%
Test Overall Accuracy: 92.01% Test mIoU : 70.06%  Test Loss: 0.1764
None : 91.35%  |  Batiment : 48.77%
